# task: create a user-user network from the raw `ratings` dataset

* load ratings file (code provided)
* generate a set of edges of the form `[(u,v,w)]` where each edge represents the fact that users `u`,`v` have both rated the same `w` movies (ie if they have both rated movie1, movie2, the edge weigth will be 2)
 * you should have 164054 edges
* save the set of edges to a file as indicated below. **Make sure you name the file using your `<id>` as indicated below, to avoid conflicts with other students within the file workspace**

In [2]:
import pandas as pd
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql import Row
from operator import add
from collections import Counter

java.net.SocketTimeoutException
	at sun.nio.ch.SocketAdaptor.connect(SocketAdaptor.java:118)
	at org.eclipse.jetty8.client.SelectConnector.startConnection(SelectConnector.java:82)
	at org.eclipse.jetty8.client.HttpDestination.startNewConnection(HttpDestination.java:292)
	at org.eclipse.jetty8.client.HttpDestination.doSend(HttpDestination.java:597)
	at org.eclipse.jetty8.client.HttpDestination.send(HttpDestination.java:532)
	at org.eclipse.jetty8.client.HttpClient.send(HttpClient.java:180)
	at com.databricks.rpc.JettyClient.$anonfun$sendRawExchange$2(JettyClient.scala:244)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:235)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:230)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:227)
	at com.databricks.rpc.JettyClient.withAttributionContext(JettyClient.scala:32)
	at com.databricks.rpc.JettyClient.sendRawExchange(JettyClient.scala:234)
	at com.databricks.rpc.JettyClient.sendExchange(JettyClient.scala:170)
	at com.databricks.rpc.JettyClient.sendAsync(JettyClient.scala:108)
	at com.databricks.rpc.BoundRPCClient.sendAsync(BoundRPCClient.scala:47)
	at com.databricks.backend.daemon.driver.DriverClient.com$databricks$backend$daemon$driver$DriverClient$$send(DriverClient.scala:95)
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$1(DriverClient.scala:283)
	at com.databricks.util.FutureUtils$.retry(FutureUtils.scala:89)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:91)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:89)
	at scala.concurrent.Future.$anonfun$recoverWith$1(Future.scala:417)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:41)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at com.databricks.threading.NamedExecutor$$anon$2.$anonfun$run$1(NamedExecutor.scala:299)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:235)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:230)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:227)
	at com.databricks.threading.NamedExecutor.withAttributionContext(NamedExecutor.scala:245)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:299)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)

In [3]:
# 1. Load ratings file (code provided)
# IN
RATINGS_SMALL_PARQUET = "/FileStore/tables/ratings-small.parquet"

# OUT
EDGES_SMALL_PARQUET = "/FileStore/tables/180571930/edges-small.parquet"

In [4]:
ratings = spark.read.parquet(RATINGS_SMALL_PARQUET)
display(ratings)

In [5]:
ratings.count()

md ### count number of distinct users  -- this is the number of unique userId that you will have in your adjacency list:

In [7]:
ratings.agg(f.countDistinct('userId')).show()

In [8]:
ratings.agg(f.countDistinct('movieId')).show()

#add your code here. 

you need to construct a RDD consisting of edges of the form:

`    [(source_node, target_node, weight)]`
    
    for example: `[(1,6,1), (1,8,1),(2,3,1), (2,4,1)]`
    
please name the RDD with edges `weightedEdges`

In [10]:
# 2. Generate a set of edges of the form [(u,v,w)]

In [11]:
# 2.1 Drop two attributes - rating, timestamp
ratings = ratings.drop('rating','timestamp')
ratings.take(10)

In [12]:
# 2.2 Convert dataframe to rdd
r = ratings.rdd
# type(r)
r.take(10)

In [13]:

# 2.3 Swap - (movieId, userId)
movie_user = r.map(lambda x: (x[1], x[0]))
movie_user.collect()

In [14]:
# 2.4 Group by key: movie, {user1, user5, ...}
output = movie_user.groupByKey().mapValues(list)
# output.count() # 9724
output.collect()

In [15]:
# 2.5 Combinate two users with the same movieId
import itertools

def combinations(row):
  k = row[0]
  return [(k, v) for v in itertools.combinations(row[1], 2)]

b = output.map(combinations).flatMap(lambda x: x)
# [(1, (1, 2)), (1, (1, 3)), (1, (1, 4)), ...]

# 2.6 Caluculate the count
c = b.map(lambda x: x[1]).countByValue()
# [(1, 2), (1, 4), (2, 4), (3, 4), (3, 5), (4, 5), (1, 2), (1, 3), (2, 3)]
# defaultdict(<class 'int'>, {(1, 2): 2, (1, 3): 1, (2, 3): 1, (4, 5): 1, (3, 4): 1, (2, 4): 1, (1, 4): 1, (3, 5): 1})

# 2.7 Split the item to [(1,2,3), ...]
d = sc.parallelize([c]).flatMap(lambda x: x.items())
# [((1, 2), 2), ((1, 3), 1),((2, 3), 1),...]

d.count() # 164054

In [16]:
d.take(10) # display the result

In [17]:
# 2.8 Flat the list and sort by source_node
e = d.map(lambda x: (x[0][0],x[0][1],x[1])).sortBy(lambda x: (x[0], x[1]))# [(266, 351, 6),...]

In [18]:
e.collect()

In [19]:
# 2.9 Rdd to dataframe
weightedEdges = sqlContext.createDataFrame(e, ['source_node', 'target_node', 'weight'])
display(weightedEdges)

In [20]:
# 3. Save the set of edges to a file as indicated below. 

In [21]:
weightedEdges.withColumnRenamed('count','weight').write.parquet(EDGES_SMALL_PARQUET, mode="overwrite")

In [22]:
r = spark.read.parquet(EDGES_SMALL_PARQUET)
display(r)

In [23]:
r.count()